In [1]:
import anndata as ad
import numpy as np
import spatialdata
import scportrait
import pandas as pd

/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/sophia/mambaforge/envs/scportrait/lib/python3.11/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [ ]:
# load scPortrait spatial data object
sdata_path = "../../input_data/CODEX_tonsil_healthy_tonsilitis/processed_data/scportrait_project_codex_region1/scportrait.sdata"
sdata = spatialdata.read_zarr(sdata_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04


In [6]:
# define dictionary for simplifying OT assigned cell types
cell_types_to_merge = {
    "cycling FDC": "FDC"
    }

simplified_celltypes = {
    "MBC": "Memory/Activated B",
    "Activated NBC": "Memory/Activated B",
    "NK": "NK/ILC",
    "ILC": "NK/ILC",
    "DC":"Dendritic cells",
    "PDC":"Dendritic cells",
    "epithelial":"Stromal",
    "FDC":"Stromal",
}

In [ ]:
# load gene expression data and OT cell type annotations
adata_region1_gex = ad.read_h5ad("generated_files/predicted_ct_tonsilitis_decoded.h5ad")
X = np.where(adata_region1_gex.obsm['X_gexp_log_whole_genome'] < 0, 0, adata_region1_gex.obsm['X_gexp_log_whole_genome']) # due to continous decoder we can get negative values, these are artefacts and should be set to 0
adata_region1_gex = ad.AnnData(X = X,
                               var = pd.DataFrame(index = adata_region1_gex.uns['var_names_whole_genome']), 
                               obs = adata_region1_gex.obs)

adata_region1_gex.obs["cell_type"] = adata_region1_gex.obs["cell_type"].replace(cell_types_to_merge).cat.remove_unused_categories()
adata_region1_gex.obs["simplified_cell_type"] = adata_region1_gex.obs["cell_type"].replace(simplified_celltypes).cat.remove_unused_categories()

adata_region1_gex.obs["region"] = "seg_all_nucleus"
adata_region1_gex.obs["region"] = adata_region1_gex.obs["region"].astype("category")
adata_region1_gex.uns = {'spatialdata_attrs': {'instance_key': 'scportrait_cell_id',
  'region': ['seg_all_nucleus'],
  'region_key': 'region'}}

# add annotating table to scportrait spatial data object and save to disk
scportrait.tl.sdata.add_element_sdata(sdata, adata_region1_gex, "complete_gene_expression") 

/var/folders/35/p4c58_4n3bb0bxnzgns1t7kh0000gn/T/ipykernel_47551/3736292599.py:7: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  adata_region1_gex.obs["cell_type"] = adata_region1_gex.obs["cell_type"].replace(cell_types_to_merge).cat.remove_unused_categories()
/var/folders/35/p4c58_4n3bb0bxnzgns1t7kh0000gn/T/ipykernel_47551/3736292599.py:8: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  adata_region1_gex.obs["simplified_cell_type"] = adata_region1_gex.obs["cell_type"].replace(simplified_celltypes).cat.remove_unused_categories()
